# 0.0. Imports

In [70]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import sqlite3


from sqlalchemy import create_engine
from umap.umap_ import UMAP
from scipy.cluster import hierarchy as hc

from sklearn import cluster
from sklearn import metrics
from sklearn import preprocessing as pp
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import ensemble as en
from sklearn.mixture import GaussianMixture as gm

## 0.2. Load Data

In [2]:
df_raw = pd.read_csv('../data/raw/Ecommerce.csv', encoding='latin1')

# drop extra column
df_raw = df_raw.drop('Unnamed: 8', axis=1)

# 1.0. Data Description

In [3]:
df1 = df_raw.copy()

## 1.1. Rename Columns

In [4]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df1.columns = cols_new

## 1.2. Data Dimnesions

In [5]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))

Number of Rows: 541909
Number of Columns: 8


## 1.3. Data Types

In [6]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [7]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [8]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [9]:
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge 
df1 = pd.merge(df1, df_backup, how='left', on='invoice_no' )

# coalesce 
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

df1 = df1.drop(['customer_id_x', 'customer_id_y'], axis=1)

## 1.6. Change Dtypes

In [10]:
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'] )

# 2.0. Data Filtering

In [11]:
df2 = df1.copy()

## 2.1. Filter Columns

In [12]:
cols_drop = ['description']
df2 = df2.drop(cols_drop, axis=1)

## 2.2. Filter Rows

In [13]:
# Numerical Attributes
df2 = df2.loc[df2['unit_price'] >= 0.4, :]

# Categorical Attributes
df2 = df2.loc[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'] ), :]

# map
df2 = df2.loc[~df2['country'].isin(['European Community', 'Unspecified' ]), :]

# bad user
df2 = df2[~df2['customer_id'].isin( [16446] )]

# quantity
df2_returns   = df2.loc[df2['quantity'] < 0, :]
df2_purchases = df2.loc[df2['quantity'] >= 0, :]

# 3.0. Feature Engineering

In [14]:
df3 = df2.copy()

## 3.1. Feature Creation

In [15]:
drop_cols = ['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country']
df_ref = df3.drop(drop_cols, axis=1).drop_duplicates(ignore_index=True)

In [16]:
df2_purchases.loc[:, ['gross_revenue']] = (df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price'])

/home/cid/.pyenv/versions/3.8.0/envs/clustering-high-value-customers-identification/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


### 3.1.1. Monetary

In [17]:
df_monetary = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index().rename(columns={'gross_revenue': 'monetary'})
df_ref = pd.merge(df_ref, df_monetary, how='left', on='customer_id')
df_ref.isna().sum()

customer_id     0
monetary       92
dtype: int64

### 3.1.2. Recency

In [18]:
df_recency = df2_purchases.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df_recency['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_ref = pd.merge(df_ref, df_recency[['customer_id', 'recency_days']], how='left', on='customer_id')

df_ref.isna().sum()

customer_id      0
monetary        92
recency_days    92
dtype: int64

### 3.1.3. Quatily of purchased

In [19]:
df_freq = df2_purchases[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().\
rename(columns={'invoice_no': 'qtde_invoices'})

df_ref = pd.merge(df_ref, df_freq, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
monetary         92
recency_days     92
qtde_invoices    92
dtype: int64

### 3.1.4. Quantity total of items purchased

In [20]:
df_freq = (df2_purchases.loc[:, ['customer_id', 'quantity']].groupby('customer_id')
                                                            .sum()
                                                            .reset_index()
                                                            .rename(columns={'quantity': 'qtde_items'}))
df_ref = pd.merge(df_ref, df_freq, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
monetary         92
recency_days     92
qtde_invoices    92
qtde_items       92
dtype: int64

### 3.1.5. Quantity of products purchased

In [21]:
df_freq = ( df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby('customer_id')
                                                               .count()
                                                               .reset_index()
                                                               .rename(columns={'stock_code': 'qtde_products'}))
df_ref = pd.merge(df_ref, df_freq, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
monetary         92
recency_days     92
qtde_invoices    92
qtde_items       92
qtde_products    92
dtype: int64

### 3.1.6. Average Ticket

In [22]:
df_avg_ticket =  (df2_purchases.loc[:, ['customer_id','gross_revenue']].groupby('customer_id')
                                                                       .mean()
                                                                       .reset_index()
                                                                       .rename(columns={'gross_revenue': 'avg_ticket'}))

df_ref = pd.merge(df_ref, df_avg_ticket, how='left', on='customer_id')

df_ref.isna().sum()

customer_id       0
monetary         92
recency_days     92
qtde_invoices    92
qtde_items       92
qtde_products    92
avg_ticket       92
dtype: int64

### 3.1.7. Average Recency Days

In [23]:
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[False, False])
df_aux['next_customer_id'] = df_aux['customer_id'].shift()
df_aux['previus_date'] = df_aux['invoice_date'].shift()

In [24]:
df_aux['avg_recency_days'] = df_aux.apply( lambda x: (x['invoice_date'] - x['previus_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)
df_aux['avg_recency_days'] = df_aux['avg_recency_days'] * -1
df_aux = df_aux.drop(columns=['invoice_date', 'next_customer_id', 'previus_date'], axis=1).dropna()

In [25]:
df_avg_recency_days = df_aux.groupby( 'customer_id' ).mean().reset_index()

df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
monetary              92
recency_days          92
qtde_invoices         92
qtde_items            92
qtde_products         92
avg_ticket            92
avg_recency_days    2815
dtype: int64

### 3.1.8. Frequency Purchase

In [26]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                 .groupby('customer_id')
                                                 .agg( max_ = ('invoice_date', 'max'),
                                                       min_ = ('invoice_date', 'min'),
                                                       days = ('invoice_date', lambda x: ((x.max() - x.min()).days) + 1 ),
                                                       buy_ = ( 'invoice_no', 'count' ))).reset_index()

df_aux['frequency'] = df_aux[['buy_', 'days']].apply( lambda x: x['buy_'] / x['days'] if x['days'] != 0 else 0, axis=1 )

df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
monetary              92
recency_days          92
qtde_invoices         92
qtde_items            92
qtde_products         92
avg_ticket            92
avg_recency_days    2815
frequency             92
dtype: int64

### 3.1.9. Number Or Returns

In [27]:
df_returns = df2_returns[['quantity', 'customer_id']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtde_returns'})
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, on='customer_id', how='left')
df_ref['qtde_returns'].fillna(0, inplace=True)
df_ref.isna().sum()

customer_id            0
monetary              92
recency_days          92
qtde_invoices         92
qtde_items            92
qtde_products         92
avg_ticket            92
avg_recency_days    2815
frequency             92
qtde_returns           0
dtype: int64

### 3.1.10. Basket Size

In [28]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                   .agg( n_purchase=('invoice_no', 'nunique'),
                                                                         n_products=('quantity', 'sum'))).reset_index()
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']
df_ref = pd.merge(df_ref, df_aux[['avg_basket_size', 'customer_id']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
monetary              92
recency_days          92
qtde_invoices         92
qtde_items            92
qtde_products         92
avg_ticket            92
avg_recency_days    2815
frequency             92
qtde_returns           0
avg_basket_size       92
dtype: int64

### 3.1.11. Unique Basket Size

In [29]:
df_aux = (df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id' )
                                                                         .agg(n_purchase=('invoice_no', 'nunique'),
                                                                              n_products=('stock_code', 'nunique'))).reset_index()
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_ref = pd.merge(df_ref, df_aux[['avg_unique_basket_size', 'customer_id']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id                  0
monetary                    92
recency_days                92
qtde_invoices               92
qtde_items                  92
qtde_products               92
avg_ticket                  92
avg_recency_days          2815
frequency                   92
qtde_returns                 0
avg_basket_size             92
avg_unique_basket_size      92
dtype: int64

# 4.0. EDA  

In [44]:
df_ref = df_ref.dropna()

df4 = df_ref[['customer_id', 'monetary', 'recency_days', 'qtde_products', 'frequency', 'qtde_returns']].copy()

## 4.3. Space Study

In [45]:
df43 = df4[['customer_id', 'monetary', 'recency_days', 'qtde_products', 'frequency', 'qtde_returns']].copy()

In [32]:
mms = pp.MinMaxScaler()

df43['monetary'] = mms.fit_transform( df43[['monetary']].values )
df43['recency_days'] = mms.fit_transform( df43[['recency_days']].values )
df43['qtde_products'] = mms.fit_transform( df43[['qtde_products']].values )
df43['frequency'] = mms.fit_transform( df43[['frequency']].values )
df43['qtde_returns'] = mms.fit_transform( df43[['qtde_returns']].values )


X = df43.copy()

### 4.3.4. Tree-Based Embbedding

In [33]:
X = df43.drop(columns=['customer_id', 'monetary'])
y = df43['monetary']

# model training
rf_model = en.RandomForestRegressor( n_estimators=100, random_state=42 )

# model training definition
rf_model.fit( X, y)

# Leaf
df_leaf = pd.DataFrame( rf_model.apply( X ) )

In [34]:
df_leaf_umap = pd.DataFrame()

# reduzer dimensionality
reducer = UMAP(random_state=42)
embedding = reducer.fit_transform( df_leaf )

# embedding
df_leaf_umap['x'] = embedding[:, 0]
df_leaf_umap['y'] = embedding[:, 1]

# # plot 
# sns.scatterplot( x='x', y='y', data=df_leaf_umap );

# 5.0. Data Preparation 

# 6.0. Feature Selection

In [35]:
df6 = df_leaf_umap.copy()

# 7.0. Hyperpameter Fine Tuning

In [36]:
# X = df6.drop(columns=['customer_id'])
X = df6.copy()

# 8.0. Model Training

In [37]:
k = 8

# model definition
gmm_model = gm(n_components=k, n_init=100, random_state=42)

# model training
gmm_model.fit(X)

# model predict
labels = gmm_model.predict(X)

# 9.0. Cluster Analysis

In [58]:
df92 = df43.copy()
df92['cluster'] = labels


# change dtypes
df92['recency_days'] = df92['recency_days'].astype(int)
df92['customer_id'] = df92['customer_id'].astype(int)
df92['qtde_products'] = df92['qtde_products'].astype(int)
df92['qtde_returns'] = df92['qtde_returns'].astype(int)

## 9.2. Cluster Profile

In [59]:
df92.head()

,customer_id,monetary,recency_days,qtde_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,0
1,13047,3232.59,56,171,0.028302,35,0
2,12583,6495.30,2,221,0.040323,50,0
3,13748,938.89,95,27,0.017921,0,4
4,15100,876.00,333,3,0.073171,22,3


In [50]:
# cluster - qt_users - per_user
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index().rename(columns={'customer_id': 'qt_users'})
df_cluster['per_user'] = 100 * (df_cluster['qt_users'] / df_cluster['qt_users'].sum())


# monetary
monetary = df92[['monetary', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, monetary, how='left', on='cluster')


# recency_days
recency_days = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, recency_days, how='left', on='cluster')


# qtde_products
qtde_products = df92[['qtde_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, qtde_products, how='left', on='cluster')


# frequency
frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, frequency, how='left', on='cluster')

# qtde_returns
qtde_returns = df92[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, qtde_returns, how='left', on='cluster')



df_cluster.sort_values('monetary', ascending=False).style.highlight_max( color='lightgreen', axis=0 )

,cluster,qt_users,per_user,monetary,recency_days,qtde_products,frequency,qtde_returns
0,0,590,19.959405,7537.835136,24.745763,357.652542,0.087172,113.132203
6,6,410,13.870095,2503.757659,56.751220,88.378049,0.057826,14.482927
5,5,329,11.129905,2437.602705,43.790274,127.355623,0.038719,19.793313
2,2,142,4.803789,2028.583239,39.626761,99.732394,0.040345,7.133803
1,1,651,22.023004,1108.411536,67.038402,46.855607,0.058335,11.861751
4,4,204,6.901218,900.411716,81.416667,28.500000,0.111439,4.284314
7,7,272,9.201624,564.829816,85.169118,11.430147,0.027752,0.952206
3,3,358,12.110961,550.485894,135.888268,16.405028,0.483529,18.117318


# 11.0. Deploy to Product

## 11.1. Insert into SQLITE

In [71]:
# create table
query_create_table_insiders = """
    CREATE TABLE insiders (
        customer_id      INTEGER,
        monetary         REAL,
        recency_days     INTEGER,
        qtde_products    INTEGER,
        frequency        REAL,
        qtde_returns     INTEGER,
        cluster          INTEGER    
    )

"""

# conn = sqlite3.connect( 'insiders_db.sqlite' )
# conn.execute( query_create_table_insiders  )
# conn.commit()
# conn.close()

# insert data
conn = create_engine('sqlite:///insiders_db.sqlite', echo=False)
df92.to_sql( 'insiders', con=conn, if_exists='append', index=False )


# select data

In [75]:
query = """
    SELECT *
    FROM insiders
"""

df = pd.read_sql_query( query, con=conn )

df.head()

,customer_id,monetary,recency_days,qtde_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,0
1,13047,3232.59,56,171,0.028302,35,0
2,12583,6495.30,2,221,0.040323,50,0
3,13748,938.89,95,27,0.017921,0,4
4,15100,876.00,333,3,0.073171,22,3


In [ ]:
## Metabase
# java -jar metabase.jar